# Evolutionary Camouflage Versus a Learning Predator Population

---

**EvoCamoVsLearnPredPop.ipynb**

August 23, 2022: this version runs “local mode” with both predator and prey running on the same machine.

(The former mode can be set with `Rube_Goldberg_mode = True`)

In [1]:
# "Rube Goldberg" mode refers to running camouflage evolution on my laptop while
# running predator vision in cloud via Colab. State is passed back and forth via
# files on Google Drive.

# TODO 20220822
# Rube_Goldberg_mode = True
Rube_Goldberg_mode = False

def if_RG_mode(for_RG_mode, for_normal_mode):
    return for_RG_mode if Rube_Goldberg_mode else for_normal_mode

# PredatorEye directory on Drive.
pe_directory = '/content/drive/My Drive/PredatorEye/'

# Shared "communication" directory on Drive.
shared_directory = if_RG_mode(pe_directory + 'evo_camo_vs_static_fcd/',
                              '/Users/cwr/camo_data/comms/')

# This was meant (20220716) to allow reading original pre-trained model from
# Google Drive, but I'll need to retrain it for M1 (Apple Silicon).
g_drive_pe_dir = ('/Users/cwr/Library/CloudStorage/' +
                  'GoogleDrive-craig.w.reynolds@gmail.com/' +
                  'My Drive/PredatorEye/')

# Directory for pre-trained Keras/TensorFlow models.
saved_model_directory = if_RG_mode(pe_directory, g_drive_pe_dir) + 'saved_models/'


print('Rube_Goldberg_mode =', Rube_Goldberg_mode)
print('shared_directory =', shared_directory)
print('saved_model_directory =', saved_model_directory)

# Pathname of pre-trained Keras/TensorFlow model
trained_model = saved_model_directory + '20220321_1711_FCD6_rc4'

# Directory on Drive for storing fine-tuning dataset.
fine_tuning_directory = shared_directory + 'fine_tuning/'

my_prefix = "find_"
other_prefix = "camo_"

my_suffix =  ".txt"
# other_suffix = ".jpeg"
other_suffix = ".png"

fcd_image_size = 1024
fcd_disk_size = 201

import time
import PIL
from pathlib import Path

from tensorflow import keras
import numpy as np
import random
import math

# %tensorflow_version 2.x
import tensorflow as tf
print('TensorFlow version:', tf.__version__)

from tensorflow.keras import backend as keras_backend
keras_backend.set_image_data_format('channels_last')

# Import DiskFind utilities for PredatorEye.
import sys
if Rube_Goldberg_mode:
    sys.path.append('/content/drive/My Drive/PredatorEye/shared_code/')
else:
    sys.path.append('/Users/cwr/Documents/code/PredatorEye/')
import DiskFind as df

Rube_Goldberg_mode = False
shared_directory = /Users/cwr/camo_data/comms/
saved_model_directory = /Users/cwr/Library/CloudStorage/GoogleDrive-craig.w.reynolds@gmail.com/My Drive/PredatorEye/saved_models/
TensorFlow version: 2.9.2


# Ad hoc “predator server”

In [2]:
# Top level: wait for camo_xxx.jpeg files to appear, respond with find_xxx.txt
def start_run(step = 0):
    if step == 0:
        print('Start run in', shared_directory )
    else:
        print('Continue run at step', step, ' in', shared_directory)
    while True:
        perform_step(step, shared_directory)
        step += 1

# Continue from from the last camo_xxx.jpeg file.
def restart_run():
    start_run(newest_file_from_other(shared_directory))

# Single step: wait for camo file, write response, delete previous response.
def perform_step(step, directory):
    wait_for_reply(step, shared_directory)
    write_response_file(step, shared_directory)
    delete_find_file(step - 1, shared_directory)

# Read image file for step, apply pre-trained model, write response file.
def write_response_file(step, directory):
    # Read image file and check for expected format.
    image_pathname = make_camo_pathname(step, directory)
    pixel_tensor = df.read_image_file_as_pixel_tensor(image_pathname)
    assert df.check_pixel_tensor(pixel_tensor), ('wrong file format: ' +
                                                 image_pathname)
    # Default Predator's model -- for prototying.
    model = Predator.default_predator().model
    # Run Predator's model on new image.
    prediction = model.predict(tf.convert_to_tensor([pixel_tensor]))[0]
    
    # Merge this step's image into fine-tuning dataset, and related bookkeeping.
    fine_tuning_dataset.update(pixel_tensor, prediction, step, directory)

    # TODO Logging, maybe only for testing, delete me?
    # Second Predator
    sp = Predator.second_predator()
    spm_predict = sp.model.predict(tf.convert_to_tensor([pixel_tensor]))[0]
    print('  "other" prediction:', spm_predict,
          ' distance to original prediction:', df.dist2d(spm_predict, prediction))
    
    # Third Predator
    tp = Predator.third_predator()
    tpm_predict = tp.model.predict(tf.convert_to_tensor([pixel_tensor]))[0]
    
    ############################################################################
    # TODO 20220902 just roughing out ordering a tournamant of 3 Predators,
    # sorting their three xy predictions. They are ordered by distance
    # from prediction to the nearest of three prey centers
    
    
#     rank_tournament(prediction_xy_a, prediction_xy_b, prediction_xy_c,
#                     prey_xy_a, prey_xy_b, prey_xy_c)

#     rank_tournament(step, directory,
#                     prediction, spm_predict, tpm_predict,
#                     0, 0, 0)


    predictions_xy3 = [prediction, spm_predict, tpm_predict]
    prey_centers_xy3 = read_3_centers_from_file(step, directory)
    ranked_predictions_xy3 = rank_tournament(predictions_xy3, prey_centers_xy3)



    
    # Generate response file.
#     response_string = (str(prediction[0])  + ' ' + str(prediction[1]) + ' ' +
#                        str(spm_predict[0]) + ' ' + str(spm_predict[1]) + ' ' +
#                        str(tpm_predict[0]) + ' ' + str(tpm_predict[1]))

    response_string = ''
    for p in ranked_predictions_xy3:
        response_string += str(p[0]) + ' ' + str(p[1]) + ' '

    ############################################################################

    verify_comms_directory_reachable()
    with open(make_find_pathname(step, directory), 'w') as file:
        file.write(response_string)
    print('Wrote ' + "'" + response_string + "'",
          'to response file', Path(make_find_pathname(step, directory)).name)

    # Predator learns from recent experience.
    print('  fine-tune default_predator:')
    Predator.default_predator().fine_tune_model(pixel_tensor, prediction,
                                                step, directory)
    # For testing/prototyping
    print('  fine-tune second_predator:')
    sp.fine_tune_model(pixel_tensor, prediction, step, directory)
    print('  fine-tune third_predator:')
    tp.fine_tune_model(pixel_tensor, prediction, step, directory)


# Delete the given file, usually after having written the next one.
def delete_find_file(step, directory):
    # Why doesn't pathlib provide a Path.remove() method like os?
    # TODO oh, missing_ok was added at pathlib version 3.8.
    # Path(makeMyPathname(step, directory)).unlink(missing_ok=True)
    p = Path(make_find_pathname(step, directory))
    if p.exists():
        p.unlink()

# Delete any remaining file in commuications directory to start a new run.
def clean_up_communication_directory():
    def delete_directory_contents(directory_path):
        for path in directory_path.iterdir():
            print('Removing from communication directory:', path)
            if path.is_dir():
                delete_directory_contents(path)
                path.rmdir()
            else:
                path.unlink()
    delete_directory_contents(Path(shared_directory))

# From pathname for file of given step number from the "other" agent.
def make_camo_pathname(step, directory):
    return directory + other_prefix + str(step) + other_suffix

# Form pathname for "find_xx.txt" response file from "this" agent.
def make_find_pathname(step, directory):
    return directory + my_prefix + str(step) + my_suffix

# Form pathname for "prey_xx.txt" ground truth file from "other" agent.
def make_prey_pathname(step, directory):
    return directory + 'prey_' + str(step) + '.txt'

# Used to ping the comms directory when it seems hung.
def write_ping_file(count, step, directory):
    pn = directory + 'ping_cloud_' + str(step) + '.txt'
    verify_comms_directory_reachable()
    with open(pn, 'w') as file:
        file.write(str(count))
    print('Ping comms: ', count, pn)

# Wait until other agent's file for given step appears.
def wait_for_reply(step, directory):
    camo_pathname = Path(make_camo_pathname(step, directory))
    camo_filename = camo_pathname.name
    prey_pathname = Path(make_prey_pathname(step, directory))
    prey_filename = prey_pathname.name
    print('Waiting for', camo_filename, 'and', prey_filename, '...',
          end='', flush=True)
    start_time = time.time()
    # Loop until both files are present, waiting 1 second between tests.
    test_count = 0
    while not (is_file_present(camo_pathname) and
               is_file_present(prey_pathname)):
        time.sleep(1)
        test_count += 1
        if (test_count % 100) == 0:
            write_ping_file(test_count, step, directory)
    print(' done, elapsed time:', int(time.time() - start_time), 'seconds.')

# Like fs::exists()
def is_file_present(file):
    result = False
    verify_comms_directory_reachable()
    filename = Path(file).name
    directory = Path(file).parent
    for i in directory.iterdir():
        if i.name == filename:
            result = True
    return result

# Returns the step number of the newest file from "other" in given directory.
# (So if "camo_573.jpeg" is the only "other" file there, returns int 573)
def newest_file_from_other(directory):
    steps = [0]  # Default to zero in case dir is empty.
    for filename in Path(directory).iterdir():
        name = filename.name
        if other_prefix == name[0:len(other_prefix)]:
            steps.append(int(name.split(".")[0].split("_")[1]))
    return max(steps)

# Read ground truth prey center location data provided in "prey_n.txt" file.
def read_3_centers_from_file(step, directory):
    # Read contents of file as string.
    verify_comms_directory_reachable()
    with open(make_prey_pathname(step, directory), 'r') as file:
        prey_centers_string = file.read()
    # Split string at whitespace, map to 6 floats, reshape into 3 xy pairs.
    # (TODO could probably be rewritten cleaner with "list comprehension")
    array = np.reshape(list(map(float, prey_centers_string.split())), (3, 2))
    return array.tolist()

# Keep log of in_disk metric.
def write_in_disk_log(step, history):
    if step % 10 == 0:
        in_disk = history.history["in_disk"][0]
        pathname = shared_directory + 'in_disk_log.csv'
        verify_comms_directory_reachable()
        with open(pathname, 'a') as file:
            if step == 0:
                file.write('step,in_disk\n')
            file.write(str(step) + ',' + "{:.4f}".format(in_disk) + '\n')

# Just wait in retry loop if shared "comms" directory become unreachable.
# Probably will return shortly, better to wait than signal a file error.
# (This is called from places with a local "directory" but it uses global value.)
def verify_comms_directory_reachable():
    seconds = 0
    # shared_directory_pathname = Path(shared_directory)
    # while not shared_directory_pathname.is_dir():
    while not Path(shared_directory).is_dir():
        print("Shared “comms” directory,", shared_directory, 
              "has been inaccessible for", seconds, "seconds.")
        time.sleep(1)  # wait 1 sec
        seconds += 1

# Given 3 prey positions ("xy3"), sort them by proximity to "point" (prediction)
def sort_xy3_by_proximity_to_point(xy3, point):
    # print('xy3 =', xy3)
    xy3_plus_distance = [[df.dist2d(xy, point), xy] for xy in xy3]
    # print('xy3_plus_distance =', xy3_plus_distance)
    sorted_xy3_plus_key = sorted(xy3_plus_distance, key=lambda x: x[0])
    # print('sorted_xy3_plus_key =', sorted_xy3_plus_key)
    sorted_xy3 = [x[1] for x in sorted_xy3_plus_key]
    # print('sorted_xy3 =', sorted_xy3)
    return sorted_xy3

# Read pre-trained model

As I integrate this into the Predator class, this is no longer “Read pre-trained model” but more like “Some utilities for reading the pre-trained model”

In [3]:
# Read pre-trained TensorFlow "predator vision" model.

# print('Reading pre-trained model from:', trained_model)

# ad hoc workaround suggested on https://stackoverflow.com/q/66408995/1991373
#
# dependencies = {
#     'hamming_loss': tfa.metrics.HammingLoss(mode="multilabel", name="hamming_loss"),
#     'attention': attention(return_sequences=True)
# }
#
# dependencies = {
#     'valid_accuracy': ValidAccuracy
# }

# Calculates RELATIVE disk radius on the fly -- rewrite later.
def fcd_disk_radius():
    return (float(fcd_disk_size) / float(fcd_image_size)) / 2

# Given two tensors of 2d point coordinates, return a tensor of the Cartesian
# distance between corresponding points in the input tensors.
def corresponding_distances(y_true, y_pred):
    true_pos_x, true_pos_y = tf.split(y_true, num_or_size_splits=2, axis=1)
    pred_pos_x, pred_pos_y = tf.split(y_pred, num_or_size_splits=2, axis=1)
    dx = true_pos_x - pred_pos_x
    dy = true_pos_y - pred_pos_y
    distances = tf.sqrt(tf.square(dx) + tf.square(dy))
    return distances

# 20211231 copied from Find_Concpocuous_Disk
def in_disk(y_true, y_pred):
    distances = corresponding_distances(y_true, y_pred)
    # relative_disk_radius = (float(fcd_disk_size) / float(fcd_image_size)) / 2

    # From https://stackoverflow.com/a/42450565/1991373
    # Boolean tensor marking where distances are less than relative_disk_radius.
    # insides = tf.less(distances, relative_disk_radius)
    insides = tf.less(distances, fcd_disk_radius())
    map_to_zero_or_one = tf.cast(insides, tf.int32)
    return map_to_zero_or_one

dependencies = { 'in_disk': in_disk }

def read_default_pre_trained_model():
    print('Reading pre-trained model from:', trained_model)
    return keras.models.load_model(trained_model, custom_objects=dependencies)

# FineTuningDataset

In [4]:
class FineTuningDataset:
    """Manages the dataset of images and labels for fine-tuning."""

    # Accumulated a new “training set” of the most recent N steps seen so far. (See
    # https://cwreynolds.github.io/TexSyn/#20220421 and ...#20220424 for discussion
    # of this parameter. Had been 1, then 100, then 200, then finally, infinity.) 
    # max_training_set_size = float('inf') # keep ALL steps in training set, use GPU.
    max_training_set_size = 500 # Try smaller again, "yellow flowers" keeps failing.
    # List of "pixel tensors".
    fine_tune_images = []
    # List of xy3 [[x,y],[x,y],[x,y]] for 3 prey centers.
    fine_tune_labels = []

    def update(self, pixel_tensor, prediction, step, directory):
        # Assume the predator was "aiming for" that one but missed by a bit.
        xy3 = read_3_centers_from_file(step, directory)
        sorted_xy3 = sort_xy3_by_proximity_to_point(xy3, prediction)

        # Accumulate the most recent "max_training_set_size" training samples.
        self.fine_tune_images.append(pixel_tensor)
        self.fine_tune_labels.append(sorted_xy3)

        # If training set has become too large, slice off first element of each.
        if len(self.fine_tune_images) > self.max_training_set_size:
            self.fine_tune_images = self.fine_tune_images[1:]
            self.fine_tune_labels = self.fine_tune_labels[1:]

        print('  fine_tune_images shape =', np.shape(self.fine_tune_images),
              '-- fine_tune_labels shape =', np.shape(self.fine_tune_labels))
        

# Create a global FineTuningDataset object.
# (TODO globals are usually a bad idea, reconsider this.)
fine_tuning_dataset = FineTuningDataset()

# Predator class

In [5]:
class Predator:
    """Represents a Predator in the camouflage simulation. It has a CNN-based
       model of visual hunting that identified the position of likely prey."""

    # Global list of active Predators. (As a class variable.)
    population = []

    def __init__(self):
        # Each Predator has its own copy of a prey-finding CNN-based model.
        self.model = []
        # By default add this new Predator to the population (TODO reconsider)
        Predator.population.append(self)

    # Set this Predator's model to default.
    def initialize_to_pre_trained_model(self):
        self.model = read_default_pre_trained_model()

    # This are presumably just for prototyping
    def default_predator():
        return Predator.population[0]
    def second_predator():
        return Predator.population[1]
    # TODO 20220901 very temp
    def third_predator():
        return Predator.population[2]

    # Keep track of how often selected prey is nearest center:
    nearest_center = 0

    # Apply fine-tuning to (originally pre-trained) predator. Use recent steps as
    # training set. Assume they were "near misses" and so training label is actual
    # (ground truth) center of disk nearest prediction. Keep a max number of old
    # steps to allow gradually forgetting the earliest part of the run.
    def fine_tune_model(self, pixel_tensor, prediction, step, directory):
        # Assume the predator was "aiming for" that one but missed by a bit.
        xy3 = read_3_centers_from_file(step, directory)
        sorted_xy3 = sort_xy3_by_proximity_to_point(xy3, prediction)

        # keep track of how often selected prey is nearest center:
        temp = xy3.copy()  # needed?
        sorted_by_dist_to_center = sort_xy3_by_proximity_to_point(temp, [0.5, 0.5])
        if sorted_by_dist_to_center[0] == sorted_xy3[0]:
            Predator.nearest_center += 1
        
        # TODO 20220829 "ad-hoc-ly" adjust for two models running, hence NC count doubled.
        # print('  nearest_center:',
        #       str(int(100 * float(self.nearest_center) / (step + 1))) + '%',
        #       '(nearest_center =', self.nearest_center, ', steps =', step + 1, ')')
        nc = self.nearest_center / 2
        print('  nearest_center:',
              str(int(100 * float(nc) / (step + 1))) + '%',
              '(nearest_center =', nc, ', steps =', step + 1, ')')

        # Convert training data list to np arrays
        images_array = np.array(fine_tuning_dataset.fine_tune_images)
        labels_array = np.array([x[0] for x in fine_tuning_dataset.fine_tune_labels])

        # print('images_array.shape =', images_array.shape,
        #       '-- labels_array.shape =', labels_array.shape)

    	# Skip fine-tuning until dataset is large enough (10% of max size).
        ########################################################################
        # TODO 20220803 clone model

        # print('disabled "Skip fine-tuning until dataset is large enough"')
        # # if images_array.shape[0] > (fine_tuning_dataset.max_training_set_size * 0.1):
        # if images_array.shape[0] > 0: ##########################################
        #     # Do fine-tuning training step using data accumulated during run.
        #     history = self.model.fit(x=images_array, y=labels_array)
        #     # Keep log of in_disk metric:
        #     write_in_disk_log(step, history)

        # print('disabled "Skip fine-tuning until dataset is large enough"')
        # # if images_array.shape[0] > (fine_tuning_dataset.max_training_set_size * 0.1):
        # if images_array.shape[0] > 0: ##########################################
        #     # Do fine-tuning training step using data accumulated during run.
        #     history = self.model.fit(x=images_array, y=labels_array)
        #     # Keep log of in_disk metric:
        #     write_in_disk_log(step, history)

        # TODO 20220823 restore "Skip fine-tuning until dataset is large enough"
        
#         print('disabled "Skip fine-tuning until dataset is large enough"')
#         if images_array.shape[0] > 0: ##########################################

        if images_array.shape[0] > (fine_tuning_dataset.max_training_set_size * 0.1):

            # # Do fine-tuning training step using data accumulated during run.
            # history = self.model.fit(x=images_array, y=labels_array)
            
            # TODO 20220823 -- run fine-tuning on CPU only.
            print('Running on CPU ONLY!')
            with tf.device('/cpu:0'):
                # Do fine-tuning training step using data accumulated during run.
                history = self.model.fit(x=images_array, y=labels_array)
            
            # Keep log of in_disk metric:
            write_in_disk_log(step, history)

        print(end='  ')
        self.print_model_trace()
        ########################################################################
    
    ############################################################################
    # TODO 20220803 clone model

    # # Copy the neural net model of a given predator into this one.
    # def copy_model(self, another_predator):
    #     # No this is wrong, just does a shallow copy
    #     # self.model = another_predator.model
    #     self.model = tf.keras.models.clone_model(another_predator.model)
    #     # Compile newly cloned model.
    #     # compile_disk_finder_model(self.model)
    #     df.compile_disk_finder_model(self.model)

    # Copy the neural net model of a given predator into this one. (From "Make
    # deep copy of keras model" https://stackoverflow.com/a/54368176/1991373)
    def copy_model(self, another_predator):
        other_model = another_predator.model
        # Clone layer structure of other model.
        # self.model = tf.keras.models.clone_model(another_predator.model)
        self.model = tf.keras.models.clone_model(other_model)
        # Compile newly cloned model.
        df.compile_disk_finder_model(self.model)

        # Copy weights of other model.
        self.model.set_weights(other_model.get_weights())

    ############################################################################

    # Modify this Predator's model by adding noise to its weights.
    def jiggle_model(self):
        weight_perturbation(self.model, 0.001)
    
    # Print the "first" weight of each layer of this Predator's Keras model.
    def print_model_trace(self):
        for layer in self.model.layers:
            trainable_weights = layer.trainable_variables
            for weight in trainable_weights:            
                weight_shape = tf.shape(weight)
                total_size = tf.math.reduce_prod(weight_shape)
                reshape_1d = tf.reshape(weight, [total_size])
                
                # TODO 20220829 take "middle" not first parameter.
                # value = reshape_1d[0].numpy()
                middle = math.floor(total_size / 2)
                value = reshape_1d[middle].numpy()
        
                print(round(value, 2), end = " ")
        print()




# Utility based on https://stackoverflow.com/a/64542651/1991373
def weight_perturbation(model, max_range):
    """Add noise to all weights in a Keras model."""
    for layer in model.layers:
        trainable_weights = layer.trainable_variables
        for weight in trainable_weights:
            random_weights = tf.random.uniform(tf.shape(weight),
                                               max_range / -2,
                                               max_range / 2,
                                               dtype=tf.float32)
            weight.assign_add(random_weights)

############################################################################
# TODO 20220803 clone model
#     This should be a utility in DiskFind.py,
#     and be called from make_disk_finder_model()

# # Compile a disk finder model.
# def compile_disk_finder_model(model):
#     # Compile with mse loss, tracking accuracy and fraction-inside-disk.
#     model.compile(loss='mse', optimizer='adam', metrics=["accuracy", df.in_disk])

# # Compile a disk finder model.
# def compile_disk_finder_model(model):
#     # Compile with mse loss, tracking accuracy and fraction-inside-disk.
#     model.compile(loss='mse', optimizer='adam', metrics=["accuracy", in_disk])

############################################################################


# Global Predator instances for prototyping.
test_predator = Predator()
test_predator.initialize_to_pre_trained_model()
second_predator = Predator()
second_predator.copy_model(test_predator)
# TODO leave second_predator an exact copy of first_predator to test equivalence
#second_predator.jiggle_model()

third_predator = Predator()
third_predator.copy_model(test_predator)

Reading pre-trained model from: /Users/cwr/Library/CloudStorage/GoogleDrive-craig.w.reynolds@gmail.com/My Drive/PredatorEye/saved_models/20220321_1711_FCD6_rc4


# Prototype rank tournament of Predators by min prediction-prey distance 


In [6]:
# TODO 20220902 just roughing out ordering a tournamant of 3 Predators,
# sorting their three xy predictions. They are ordered by distance
# from prediction to the nearest of three prey centers (smaller being
# earlier in sort order -- think of it as "aiming error")

# TODO 20220902 search key response_string

# TODO 20220902 does it make sense for "step" and "directory" to be parameters?

# TODO 20220902 prey position args ignored for now. Should pass in
# the xy3 of prey centers from read_3_centers_from_file()

# def rank_tournament(step, directory,
#                     prediction_xy_a, prediction_xy_b, prediction_xy_c,
#                     prey_xy_a, prey_xy_b, prey_xy_c):
#     xy3 = read_3_centers_from_file(step, directory)
    
#     # prints eg: xy3 = [[0.775153, 0.648901], [0.257899, 0.328082], [0.533892, 0.197266]]
#     print('in rank_tournament(), xy3 =', xy3)


# Use yesterday's random seed, nice patterns
# texsyn ~/Pictures/camouflage_backgrounds/tiger_eye_beans ~/Desktop/TexSyn_temp/ 0.2 20220902 512 512


# TODO 20220902 I need something sort of like this, to sort the
#               three XY positions by min distance to any prey
    
# # Given 3 prey positions ("xy3"), sort them by proximity to "point" (prediction)
# def sort_xy3_by_proximity_to_point(xy3, point):
#     # print('xy3 =', xy3)
#     xy3_plus_distance = [[df.dist2d(xy, point), xy] for xy in xy3]
#     # print('xy3_plus_distance =', xy3_plus_distance)
#     sorted_xy3_plus_key = sorted(xy3_plus_distance, key=lambda x: x[0])
#     # print('sorted_xy3_plus_key =', sorted_xy3_plus_key)
#     sorted_xy3 = [x[1] for x in sorted_xy3_plus_key]
#     # print('sorted_xy3 =', sorted_xy3)
#     return sorted_xy3


#     predictions_xy3 = [prediction, spm_predict, tpm_predict]
#     prey_centers_xy3 = read_3_centers_from_file(step, directory)
#     ranked_predictions_xy3 = rank_tournament(predictions_xy3, prey_centers_xy3)

# def rank_tournament(predictions_xy3, prey_centers_xy3):
    
#     # prints eg: prey_centers_xy3 = [[0.775153, 0.648901], [0.257899, 0.328082], [0.533892, 0.197266]]
#     print('in rank_tournament():')
#     print('    predictions_xy3 =', predictions_xy3)
#     print('    prey_centers_xy3 =', prey_centers_xy3)

#     ranked_predictions_xy3 = sort_xy3_by_least_aim_error(predictions_xy3, prey_centers_xy3)
#     return ranked_predictions_xy3

def rank_tournament(predictions_xy3, prey_centers_xy3):
    return sort_xy3_by_least_aim_error(predictions_xy3, prey_centers_xy3)




################################################################################
# Given 3 predator prediction positions ("xy3"), sort them by "least aim error",
# that is, the distance from the prediction to the nearest prey position, so
# smallest errors sort to the front of the collection.


# def sort_xy3_by_least_aim_error(predictions_xy3, prey_centers_xy3):
#     # print('xy3 =', xy3)
#     # xy3_plus_distance = [[df.dist2d(xy, point), xy] for xy in xy3]
#     predictions_with_key = [[aim_error(xy, prey_centers_xy3), xy]
#                             for xy in predictions_xy3]
#     print('predictions_with_key =', predictions_with_key)
    
#     # print('xy3_plus_distance =', xy3_plus_distance)
#     # sorted_xy3_plus_key = sorted(xy3_plus_distance, key=lambda x: x[0])
#     sorted_predictions_with_key = sorted(predictions_with_key,
#                                          key=lambda x: x[0])
#     print('sorted_predictions_with_key =', sorted_predictions_with_key)


    
#     # print('sorted_xy3_plus_key =', sorted_xy3_plus_key)
#     # sorted_xy3 = [x[1] for x in sorted_xy3_plus_key]
#     sorted_predictions = [x[1] for x in sorted_predictions_with_key]
#     print('sorted_predictions =', sorted_predictions)


#     # print('sorted_xy3 =', sorted_xy3)
#     # return sorted_xy3
#     return sorted_predictions

def sort_xy3_by_least_aim_error(predictions_xy3, prey_centers_xy3):
    predictions_with_key = [[aim_error(xy, prey_centers_xy3), xy]
                            for xy in predictions_xy3]
    # print('predictions_with_key =', predictions_with_key)
    sorted_predictions_with_key = sorted(predictions_with_key,
                                         key=lambda x: x[0])
    # print('sorted_predictions_with_key =', sorted_predictions_with_key)
    sorted_predictions = [x[1] for x in sorted_predictions_with_key]
    # print('sorted_predictions =', sorted_predictions)
    return sorted_predictions



def aim_error(prediction_xy, prey_centers_xy3):
    min_aim_error = math.inf
    for xy in prey_centers_xy3:
        distance = df.dist2d(xy, prediction_xy)
        if min_aim_error > distance:
            min_aim_error = distance
    return min_aim_error



rank_tournament([[0.2, 0.5], [0.4, 0.5], [0.3, 0.5]],
                [[0.5, 0.5], [0.6, 0.5], [0.7, 0.5]])

    



[[0.4, 0.5], [0.3, 0.5], [0.2, 0.5]]

# Run test

In [7]:
# # TODO 20220827 testing print_model_trace
# test_predator.print_model_trace()



# Keep track of how often selected prey is nearest center:
Predator.nearest_center = 0

# Predator.population = []
# TODO maybe a Predator.reset() method?

# Flush out obsolete files in comms directory.
clean_up_communication_directory()

# Start fresh run defaulting to step 0.
start_run()

Removing from communication directory: /Users/cwr/camo_data/comms/prey_176.txt
Removing from communication directory: /Users/cwr/camo_data/comms/prey_175.txt
Removing from communication directory: /Users/cwr/camo_data/comms/in_disk_log.csv
Removing from communication directory: /Users/cwr/camo_data/comms/camo_176.png
Removing from communication directory: /Users/cwr/camo_data/comms/find_176.txt
Removing from communication directory: /Users/cwr/camo_data/comms/find_175.txt
Start run in /Users/cwr/camo_data/comms/
Waiting for camo_0.png and prey_0.txt ... done, elapsed time: 16 seconds.
1/1 [==============================] - 0s 88ms/step
  fine_tune_images shape = (1, 128, 128, 3) -- fine_tune_labels shape = (1, 3, 2)
1/1 [==============================] - 0s 60ms/step
  "other" prediction: [0.83753645 0.5500108 ]  distance to original prediction: 0.0
1/1 [==============================] - 0s 58ms/step
Wrote '0.83753645 0.5500108 0.83753645 0.5500108 0.83753645 0.5500108 ' to response fi

  -0.39 -0.01 -0.11 -0.97 -0.19 -0.09 0.1 0.01 -0.01 -3.86 0.29 1.05 -0.16 -5.44 -0.02 6.25 0.03 0.21 
  fine-tune third_predator:
  nearest_center: 60% (nearest_center = 3.0 , steps = 5 )
disabled "Skip fine-tuning until dataset is large enough"
Running on CPU ONLY!
1/1 [==============================] - 0s 40ms/step - loss: 0.0273 - accuracy: 1.0000 - in_disk: 0.6000
  -0.39 -0.01 -0.11 -0.97 -0.19 -0.09 0.11 0.01 -0.01 -3.86 0.29 1.05 -0.16 -5.44 -0.02 6.25 0.03 0.21 
Waiting for camo_5.png and prey_5.txt ... done, elapsed time: 1 seconds.
1/1 [==============================] - 0s 8ms/step
  fine_tune_images shape = (6, 128, 128, 3) -- fine_tune_labels shape = (6, 3, 2)
1/1 [==============================] - 0s 9ms/step
  "other" prediction: [0.3097395  0.55526555]  distance to original prediction: 0.032076817
1/1 [==============================] - 0s 8ms/step
Wrote '0.3097395 0.55526555 0.30554828 0.58706737 0.35469845 0.62346345 ' to response file find_5.txt
  fine-tune default_pr

1/1 [==============================] - 0s 69ms/step - loss: 0.0253 - accuracy: 0.9000 - in_disk: 0.3000
  -0.39 -0.01 -0.11 -0.97 -0.19 -0.09 0.11 0.01 -0.01 -3.86 0.29 1.05 -0.16 -5.44 -0.02 6.25 0.03 0.21 
Waiting for camo_10.png and prey_10.txt ... done, elapsed time: 2 seconds.
1/1 [==============================] - 0s 9ms/step
  fine_tune_images shape = (11, 128, 128, 3) -- fine_tune_labels shape = (11, 3, 2)
1/1 [==============================] - 0s 9ms/step
  "other" prediction: [0.46348354 0.20258185]  distance to original prediction: 0.07871493
1/1 [==============================] - 0s 9ms/step
Wrote '0.46348354 0.20258185 0.54174924 0.21097954 0.43291426 0.22795324 ' to response file find_10.txt
  fine-tune default_predator:
  nearest_center: 40% (nearest_center = 4.5 , steps = 11 )
disabled "Skip fine-tuning until dataset is large enough"
Running on CPU ONLY!
1/1 [==============================] - 0s 98ms/step - loss: 0.0331 - accuracy: 0.9091 - in_disk: 0.5455
  -0.4 -0.01 

1/1 [==============================] - 0s 96ms/step - loss: 0.0290 - accuracy: 0.7333 - in_disk: 0.2000
  -0.39 -0.01 -0.11 -0.97 -0.19 -0.09 0.11 0.01 -0.01 -3.86 0.29 1.05 -0.16 -5.44 -0.02 6.25 0.03 0.21 
Waiting for camo_15.png and prey_15.txt ... done, elapsed time: 1 seconds.
1/1 [==============================] - 0s 9ms/step
  fine_tune_images shape = (16, 128, 128, 3) -- fine_tune_labels shape = (16, 3, 2)
1/1 [==============================] - 0s 8ms/step
  "other" prediction: [0.25274494 0.38918722]  distance to original prediction: 0.052456826
1/1 [==============================] - 0s 8ms/step
Wrote '0.22747925 0.43515855 0.25274494 0.38918722 0.33784825 0.46781796 ' to response file find_15.txt
  fine-tune default_predator:
  nearest_center: 46% (nearest_center = 7.5 , steps = 16 )
disabled "Skip fine-tuning until dataset is large enough"
Running on CPU ONLY!
1/1 [==============================] - 0s 145ms/step - loss: 0.0176 - accuracy: 0.8750 - in_disk: 0.4375
  -0.4 -0.0

1/1 [==============================] - 0s 120ms/step - loss: 0.0173 - accuracy: 0.8500 - in_disk: 0.2000
  -0.39 -0.01 -0.11 -0.97 -0.19 -0.09 0.11 0.01 -0.01 -3.86 0.29 1.04 -0.16 -5.44 -0.02 6.25 0.03 0.21 
Waiting for camo_20.png and prey_20.txt ... done, elapsed time: 0 seconds.
1/1 [==============================] - 0s 9ms/step
  fine_tune_images shape = (21, 128, 128, 3) -- fine_tune_labels shape = (21, 3, 2)
1/1 [==============================] - 0s 9ms/step
  "other" prediction: [0.60214585 0.48059487]  distance to original prediction: 0.04324174
1/1 [==============================] - 0s 10ms/step
Wrote '0.612712 0.43866393 0.60214585 0.48059487 0.58614117 0.4730283 ' to response file find_20.txt
  fine-tune default_predator:
  nearest_center: 45% (nearest_center = 9.5 , steps = 21 )
disabled "Skip fine-tuning until dataset is large enough"
Running on CPU ONLY!
1/1 [==============================] - 0s 123ms/step - loss: 0.0075 - accuracy: 0.9524 - in_disk: 0.5714
  -0.4 -0.01 

...

4/4 [==============================] - 1s 153ms/step - loss: 0.0092 - accuracy: 0.9100 - in_disk: 0.6100
  -0.4 -0.0 -0.11 -0.96 -0.19 -0.09 0.1 0.01 -0.01 -3.86 0.29 1.04 -0.16 -5.44 -0.02 6.26 0.02 0.22 
Waiting for camo_100.png and prey_100.txt ... done, elapsed time: 0 seconds.
1/1 [==============================] - 0s 9ms/step
  fine_tune_images shape = (101, 128, 128, 3) -- fine_tune_labels shape = (101, 3, 2)
1/1 [==============================] - 0s 8ms/step
  "other" prediction: [0.68134755 0.47918528]  distance to original prediction: 0.007922929
1/1 [==============================] - 0s 8ms/step
Wrote '0.71249396 0.48393446 0.68134755 0.47918528 0.67709416 0.4858697 ' to response file find_100.txt
  fine-tune default_predator:
  nearest_center: 53% (nearest_center = 54.0 , steps = 101 )
disabled "Skip fine-tuning until dataset is large enough"
Running on CPU ONLY!
4/4 [==============================] - 1s 145ms/step - loss: 0.0082 - accuracy: 0.9406 - in_disk: 0.6337
  -0.41

4/4 [==============================] - 1s 153ms/step - loss: 0.0152 - accuracy: 0.8762 - in_disk: 0.4476
  -0.4 -0.0 -0.11 -0.97 -0.19 -0.09 0.1 0.01 -0.01 -3.86 0.29 1.04 -0.16 -5.44 -0.02 6.26 0.02 0.23 
Waiting for camo_105.png and prey_105.txt ... done, elapsed time: 0 seconds.
1/1 [==============================] - 0s 8ms/step
  fine_tune_images shape = (106, 128, 128, 3) -- fine_tune_labels shape = (106, 3, 2)
1/1 [==============================] - 0s 8ms/step
  "other" prediction: [0.28849393 0.30183777]  distance to original prediction: 0.01262532
1/1 [==============================] - 0s 9ms/step
Wrote '0.28849393 0.30183777 0.2827632 0.31308755 0.30303913 0.3634737 ' to response file find_105.txt
  fine-tune default_predator:
  nearest_center: 50% (nearest_center = 54.0 , steps = 106 )
disabled "Skip fine-tuning until dataset is large enough"
Running on CPU ONLY!
4/4 [==============================] - 1s 158ms/step - loss: 0.0078 - accuracy: 0.9528 - in_disk: 0.6038
  -0.41 -

4/4 [==============================] - 1s 169ms/step - loss: 0.0128 - accuracy: 0.8909 - in_disk: 0.5727
  -0.4 -0.01 -0.11 -0.96 -0.18 -0.09 0.1 0.01 -0.01 -3.86 0.29 1.04 -0.16 -5.44 -0.02 6.26 0.02 0.23 
Waiting for camo_110.png and prey_110.txt ... done, elapsed time: 0 seconds.
1/1 [==============================] - 0s 9ms/step
  fine_tune_images shape = (111, 128, 128, 3) -- fine_tune_labels shape = (111, 3, 2)
1/1 [==============================] - 0s 8ms/step
  "other" prediction: [0.56880254 0.5467577 ]  distance to original prediction: 0.2031582
1/1 [==============================] - 0s 9ms/step
Wrote '0.37537044 0.48465067 0.56880254 0.5467577 0.47787714 0.5648302 ' to response file find_110.txt
  fine-tune default_predator:
  nearest_center: 50% (nearest_center = 55.5 , steps = 111 )
disabled "Skip fine-tuning until dataset is large enough"
Running on CPU ONLY!
4/4 [==============================] - 1s 167ms/step - loss: 0.0079 - accuracy: 0.8919 - in_disk: 0.7117
  -0.41 -

4/4 [==============================] - 1s 172ms/step - loss: 0.0088 - accuracy: 0.9130 - in_disk: 0.6261
  -0.4 -0.01 -0.11 -0.96 -0.18 -0.1 0.1 0.01 -0.01 -3.86 0.29 1.04 -0.16 -5.44 -0.02 6.26 0.02 0.23 
Waiting for camo_115.png and prey_115.txt ... done, elapsed time: 0 seconds.
1/1 [==============================] - 0s 8ms/step
  fine_tune_images shape = (116, 128, 128, 3) -- fine_tune_labels shape = (116, 3, 2)
1/1 [==============================] - 0s 8ms/step
  "other" prediction: [0.38001502 0.26972798]  distance to original prediction: 0.027105449
1/1 [==============================] - 0s 9ms/step
Wrote '0.39607227 0.24789062 0.38081226 0.23602027 0.38001502 0.26972798 ' to response file find_115.txt
  fine-tune default_predator:
  nearest_center: 50% (nearest_center = 58.5 , steps = 116 )
disabled "Skip fine-tuning until dataset is large enough"
Running on CPU ONLY!
4/4 [==============================] - 1s 176ms/step - loss: 0.0083 - accuracy: 0.9052 - in_disk: 0.6983
  -0.4

KeyboardInterrupt: 

In [ ]:
# Normally start from step 0, or if an "other" file exists
# (eg 'camo_123.jpeg') then restart from that point.

# restart_run()

#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#